In [ ]:
# Parte 1 - creo dataframe
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
from MJP.majority_portfolio_utilities_TV import * 

FILENAME = 'monthly_data.csv'
factors_df=pd.DataFrame({
    'factors': ['ag','beta','bm','cumret','dolvol6','gp','ill6','ns','size','volatility','acc'],
    'signs': [  -1.,  1.,   1.,    1.,     -1.,     1.,    1.,  -1.,  -1.,   -1.,   -1.],
    'wsigns':[  -1.,  1.,   1.,    1.,     -1.,     1.,    1.,  -1.,  -1.,   -1.,   -1.]
})

DATA_INIZIO = '2000-01-01'      # In futuro considerare periodi bull e bear
DATA_FINE = '2023-12-31'        # Quindi utilizzare sub periodi

df = create_df(FILENAME, factors_df['factors'].tolist(), DATA_INIZIO, DATA_FINE) # funzione sua
df['date']=pd.to_datetime(df['date']) 
print(f"DataFrame caricato: {df.shape[0]:,} righe, {df.shape[1]} colonne")
print(df.columns.tolist())

# PARTE 2 - NORMALIZZO CON QUANTILE NORMALIZER

from sklearn.preprocessing import QuantileTransformer

factors_to_normalize = ['ns', 'size', 'ag', 'gp', 'acc', 'bm', 'ill6', 'volatility', 'beta', 'cumret', 'dolvol6']

# Elimina righe con NaN solo nelle colonne da normalizzare
df_clean = df.dropna(subset=factors_to_normalize)

# Applica Quantile normalization solo ai fattori
scaler = QuantileTransformer(output_distribution='uniform', random_state=0)
df_norm = df_clean.copy()
df_norm[factors_to_normalize] = scaler.fit_transform(df_clean[factors_to_normalize])

print("✅ df normalizzato su fattori:")
df_norm.head()

In [ ]:
# Parte 3 - inizializzazione della configurazione

# *** ATTENZIONE *** --> PRIMA MODIFICA, HO MESSO TRUE A WEIGHT

# Lista dei fattori
factors_list = factors_df['factors'].to_list()
# Segni di default
default_signs = factors_df['signs'].to_list()

# Dizionario pairwise per AHP/test (tutti a 1.0 per prova)
pairwise_dict = {}
for i in range(len(factors_list)):
    for j in range(i+1, len(factors_list)):
        pairwise_dict[(factors_list[i], factors_list[j])] = 1.0  # tutti uguali
        # il reciproco viene gestito nella funzione AHP automaticamente

remove_outliers = False
inf, sup = None, None

MJ_configuration = {
    'K': 12,  # holding periods in months
    'lag': 5,  # 5 per ribilanciamento a giugno
    'factors': factors_list,
    'default_signs': default_signs,  # 1=beneficio, -1=costo
    'num_port': 10,  # number of portfolios
    'num_cat': 6,  # number of categories per factor per MJ
    'weighting': True,
    'verbose': True,
    'n_jobs': -1,
    'mj_window': 1,
    'method': 'majority',  # mean_rank, majority, 75q, 90q, lex, dlex
    'rolling_method': 'profile',  # 'rank','vote','profile'
    'treat_na_mj': 'median',
    'remove_outliers': remove_outliers,
    'inclusive': True,
    'fix_signs': True,
    'all_voters_not_nan_on_reallocation': True,
    # Parametri utilizzati solo se fix_signs=False
    'min_voters': 5,
    'voting_window': 6,
    'sign_voting_window': 12,
    'p_threshold': 0.1,
    'delta_utility': 0,
    'eliminations': 1,
    'players_batch_size': 5,
    'small': True,
    # Parametro aggiuntivo per test AHP/EQUALE WEIGHTS
    'pairwise': pairwise_dict
}

if remove_outliers:
    MJ_configuration['outliers']=[inf,sup]

if MJ_configuration['weighting']:                                                   # CHIEDEREEEEEEEEEEEEEE
    MJ_configuration['default_voters']=factors_df['factors'].to_list()
    MJ_configuration['default_signs']=factors_df['wsigns'].to_list()
    compute='wmj'
elif not MJ_configuration['weighting']:
    MJ_configuration['default_voters']=factors_df['factors'].to_list()
    MJ_configuration['default_signs']=factors_df['signs'].to_list()
    compute='mj'

factors=factors_df['factors'].to_list() # salva separatamente la lista dei nomi dei fattori 

In [ ]:
# Parte 4 - creazione portafogli single criteria e MJ

### COMPUTE SINGLE FACTOR STRATEGIES
portfolios, weighted_portfolios, portfolios_stock_reallocation = compute_factor_strategies(df_norm, MJ_configuration)
        # '2020-06-30': [list of tickers]       # Con pesi, uguale ma 'AAPL': 0.1           # 'AAPL': 2020-06-30': 'P3', '2021-06-30': 'P5'}
        # '2021-06-30': [list of tickers]
        # _ perché non considero weighted_portfolios al momento

### COMPUTE EQUAL WEIGHTED MJ STRATEGY
MJ_configuration['weighting']=False
MJ_portfolios, \
mj_voters, \
MJ_portfolios_stock_reallocation = compute_MJ_portfolio_strategy(df_norm, MJ_configuration)

portfolios['mj'] = MJ_portfolios['mj']
portfolios_stock_reallocation['mj'] = {}
portfolios_stock_reallocation['mj']['EW_turnover'] = MJ_portfolios_stock_reallocation['mj']

### COMPUTE VALUE WEIGHTED MJ STRATEGY
MJ_configuration['weighting']=True
MJ_weighted_portfolios, \
mj_voters, \
MJ_weighted_portfolios_stock_reallocation = compute_MJ_portfolio_strategy(df_norm, MJ_configuration)

weighted_portfolios['wmj'] = MJ_weighted_portfolios['wmj']
portfolios_stock_reallocation['wmj'] = {}
portfolios_stock_reallocation['wmj']['VW_turnover'] = MJ_weighted_portfolios_stock_reallocation['wmj']

'''
OUTPUT: 
portfolios = {
  'ns': DataFrame con port1...port10 costruiti su ns,
  'beta': ...,
  'bm': ...,
  ...
  'mj': DataFrame con port1...port10 creati con MJ
}
'''


In [ ]:
# ELECTRE TRI-B - VERSIONE COMPLETA E CORRETTA
from pyDecision.algorithm import electre_tri_b
from tqdm import tqdm
from pandas.tseries.offsets import MonthEnd
import numpy as np
import pandas as pd


# ------------------------
# CONFIGURAZIONE ELECTRE TRI-B
# ------------------------

num_factors = len(factors_list)
num_port = 10 

ELECTRE_configuration = {
    'K': 12,
    'lag': 5,
    'factors': factors_list,
    'default_signs': default_signs,
    'num_port': num_port,
    'weighting': False,
    'verbose': True,
    
    # Parametri ELECTRE Tri-B
    'weights': np.ones(num_factors) / num_factors,          # cambia solo se hai motivo solido
    'lambda_cut': 0.65,                                      # con molti criteri, dovrebbe stare tra 0.65 < x < 0.75
    'q_multiplier': 0.1,                                    # soglia di indifferenza, due stocks vengono considerati uguali (forse 0.2 troppo grande)
    'p_multiplier': 0.4,                                    # soglia di preferenza, maggiore di q (0.5 ok)
    'v_multiplier': 2.5,                                    # soglia di veto (almeno 3x p), vuol dire che un criterio fa da veto per bloccare relazione di outranking
    'num_profiles': num_port - 1,
    'rule': 'oc'                                            # 'oc' o 'cc', con oc = outranking congiunto e cc = outranking concordante
}


# ------------------------
# FUNZIONE ELECTRE TRI-B
# ------------------------
def compute_electre_iii_rolling_strategy(df: pd.DataFrame, configuration: dict, weighting: bool = False):
    """
    ELECTRE Tri-B implementation for portfolio sorting.
    Port 10 = BEST, Port 1 = WORST
    """
    df = df.copy()
    
    if 'medate' not in df.columns:
        df['medate'] = df['date'] + MonthEnd(0)

    # Extract parameters
    K = configuration['K']
    lag = configuration['lag']
    num_port = configuration['num_port']
    factors = sorted(configuration['factors'])
    types = configuration['default_signs']
    verbose = configuration['verbose']
    num_profiles = configuration['num_profiles']

    weights = np.array(configuration['weights'])
    lambda_cut = configuration['lambda_cut']
    q_mult = configuration['q_multiplier']
    p_mult = configuration['p_multiplier']
    v_mult = configuration['v_multiplier']
    rule = configuration['rule']
    
    criteria_type = ['max' if t == 1 else 'min' for t in types]

    # Rebalancing dates
    reallocation_dates = []
    date_start = df['medate'].min() + MonthEnd(lag)
    date_end = df['medate'].max()
    while date_start + MonthEnd(K) <= date_end:
        reallocation_dates.append(date_start)
        date_start += MonthEnd(K)

    print(f"\n{'='*70}")
    print(f"📊 ELECTRE TRI-B PORTFOLIO STRATEGY")
    print(f"{'='*70}")
    print(f"Method: ELECTRE Tri-B with {rule.upper()} assignment rule")
    print(f"Profiles: {num_profiles} boundary profiles (quantile-based)")
    print(f"Lambda cut: {lambda_cut} | Thresholds: q={q_mult}σ, p={p_mult}σ, v={v_mult}σ")
    print(f"Portfolio convention: Port 10 = BEST, Port 1 = WORST")
    print(f"Rebalancing dates: {len(reallocation_dates)}")
    print(f"{'='*70}\n")

    # Initialize
    returns_dict = {f'port{i+1}': [] for i in range(num_port)}
    returns_dict['long_short'] = []
    turnover_values = {f'port{i+1}': [] for i in range(num_port)}
    reallocation = {f'port{i+1}': {} for i in range(num_port)}
    prev_portfolios = {f'port{i+1}': None for i in range(num_port)}
    
    labels = [f'port{i+1}' for i in range(num_port)]

    # Statistics tracking
    total_assets = 0
    category_counts = {i: 0 for i in range(num_profiles + 1)}  # 0 to num_profiles

    # Rolling window loop
    iteration = 0
    for date in tqdm(reallocation_dates, desc="ELECTRE Tri-B rolling", disable=not verbose):
        iteration += 1
        
        df_now = df[df['medate'] == date].dropna(subset=factors).copy()
        
        if df_now.empty or len(df_now) < num_port:
            continue

        matrix = df_now[factors].values
        total_assets += len(df_now)
        
        # Define boundary profiles using quantiles
        percentiles = np.linspace(1.0/(num_profiles+1), num_profiles/(num_profiles+1), num_profiles)
        profiles_list = []
        for p in percentiles:
            row_profile = df_now[factors].quantile(p).values 
            profiles_list.append(row_profile)
        profiles = np.array(profiles_list)

        # Compute dynamic thresholds
        std_devs = df_now[factors].std().values
        if std_devs.ndim > 1:
            std_devs = std_devs.flatten()

        q_array = q_mult * std_devs
        p_array = p_mult * std_devs
        v_array = v_mult * std_devs

        q_vec = [float(x) for x in q_array]
        p_vec = [float(x) for x in p_array]
        v_vec = [float(x) for x in v_array]

        # Handle criteria direction
        matrix_adjusted = matrix.copy()
        profiles_adjusted = profiles.copy()
        
        for idx, crit_type in enumerate(criteria_type):
            if crit_type == 'min':
                matrix_adjusted[:, idx] = -matrix_adjusted[:, idx]
                profiles_adjusted[:, idx] = -profiles_adjusted[:, idx]

        # Execute ELECTRE Tri-B
        try:
            assignments = electre_tri_b(
                matrix_adjusted,
                weights.tolist(),
                q_vec,
                p_vec,
                v_vec,
                profiles_adjusted.tolist(),
                lambda_cut,
                False,
                rule
            )
            
            # Track category distribution
            unique_cats, counts = np.unique(assignments, return_counts=True)
            for cat, count in zip(unique_cats, counts):
                category_counts[int(cat)] += count
            
            if iteration == 1:
                print(f"First iteration ({date}):")
                print(f"  Assets: {len(df_now)}")
                print(f"  Category distribution:")
                for cat, count in zip(unique_cats, counts):
                    pct = count/len(assignments)*100
                    print(f"    Category {int(cat)}: {count} assets ({pct:.1f}%)")
            
        except Exception as e:
            print(f"\n❌ ELECTRE Tri-B error at {date}: {e}")
            continue
        
        df_now['category'] = assignments

        # ✅ MAPPATURA CORRETTA
        # ELECTRE Tri-B con 9 profili restituisce categorie: 0, 1, 2, ..., 9
        # Mappatura: Cat 0 → Port 1 (worst), Cat 1 → Port 2, ..., Cat 9 → Port 10 (best)
        
        def map_category_to_portfolio(cat):
            if pd.isna(cat):
                return np.nan
            
            cat_int = int(cat)
            
            # Category 0 (rejected) → Port 1 (worst)
            if cat_int == 0:
                return 'port1'
            
            # Valid categories: 1 to num_profiles (1 to 9)
            if cat_int < 1 or cat_int > num_profiles:
                return np.nan
            
            # Map: Cat 1 → Port 2, Cat 2 → Port 3, ..., Cat 9 → Port 10
            port_num = cat_int + 1
            return f'port{port_num}'

        df_now['portfolio'] = df_now['category'].apply(map_category_to_portfolio)
        
        if iteration == 1:
            print(f"\n  Portfolio assignment:")
            for port in labels:
                count = len(df_now[df_now['portfolio'] == port])
                if count > 0:
                    print(f"    {port}: {count} assets")
            print()
        
        df_now = df_now.dropna(subset=['portfolio'])
        
        if df_now.empty:
            continue

        # Compute returns
        period_end = date + MonthEnd(K)
        df_hold = df[(df['medate'] > date) & (df['medate'] <= period_end)]

        port_permnos = {}
        for port in labels:
            tickers = df_now[df_now['portfolio'] == port]['PERMNO'].tolist()
            port_permnos[port] = tickers
            reallocation[port][date] = tickers
            
            df_port = df_hold[df_hold['PERMNO'].isin(tickers)].copy()
            
            if df_port.empty:
                continue
            
            if weighting:
                df_port['wret'] = df_port['RET_RF'] * df_port['me_lag']
                wret_sum = df_port.groupby('medate')['wret'].sum()
                me_sum = df_port.groupby('medate')['me_lag'].sum()
                ret = (wret_sum / me_sum).dropna()
            else:
                ret = df_port.groupby('medate')['RET_RF'].mean()
            
            if isinstance(ret, pd.DataFrame):
                ret = ret.squeeze()
            
            if len(ret) > 0:
                returns_dict[port].append(ret)

        # Turnover
        for port in labels:
            curr_holdings = set(port_permnos[port])
            if prev_portfolios[port] is not None:
                prev_holdings = set(prev_portfolios[port])
                intersection = len(prev_holdings & curr_holdings)
                total_unique = max(len(prev_holdings), len(curr_holdings))
                turnover_rate = 1 - (intersection / total_unique) if total_unique > 0 else 0.0
            else:
                turnover_rate = np.nan
            turnover_values[port].append(turnover_rate)
            prev_portfolios[port] = port_permnos[port]

        # Long-short: Port 10 (best) LONG, Port 1 (worst) SHORT
        long = df_hold[df_hold['PERMNO'].isin(port_permnos['port10'])].copy()
        short = df_hold[df_hold['PERMNO'].isin(port_permnos['port1'])].copy()

        if not long.empty and not short.empty:
            if weighting:
                long['wret'] = long['RET_RF'] * long['me_lag']
                short['wret'] = short['RET_RF'] * short['me_lag']
                
                long_wret_sum = long.groupby('medate')['wret'].sum()
                long_me_sum = long.groupby('medate')['me_lag'].sum()
                ret_long = (long_wret_sum / long_me_sum).dropna()
                
                short_wret_sum = short.groupby('medate')['wret'].sum()
                short_me_sum = short.groupby('medate')['me_lag'].sum()
                ret_short = (short_wret_sum / short_me_sum).dropna()
            else:
                ret_long = long.groupby('medate')['RET_RF'].mean()
                ret_short = short.groupby('medate')['RET_RF'].mean()
            
            ls_ret = (ret_long - ret_short).dropna()
            
            if isinstance(ls_ret, pd.DataFrame):
                ls_ret = ls_ret.squeeze()
            
            if len(ls_ret) > 0:
                returns_dict['long_short'].append(ls_ret)

    # Final statistics
    print(f"\n{'='*70}")
    print(f"📊 ELECTRE TRI-B CLASSIFICATION SUMMARY")
    print(f"{'='*70}")
    print(f"Total assets classified: {total_assets:,}")
    print(f"\nCategory distribution across all periods:")
    for cat in sorted(category_counts.keys()):
        count = category_counts[cat]
        if count > 0:
            pct = count / total_assets * 100
            cat_label = "Rejected" if cat == 0 else f"Cat {cat}"
            print(f"  {cat_label}: {count:,} ({pct:.1f}%)")
    
    rejected_pct = category_counts[0] / total_assets * 100 if total_assets > 0 else 0
    print(f"\n⚠️  Rejection rate (Category 0 → Port 1): {rejected_pct:.2f}%")
    print(f"{'='*70}\n")

    # Combine results
    for key in returns_dict:
        if returns_dict[key]:
            concatenated = pd.concat(returns_dict[key], axis=0)
            if isinstance(concatenated, pd.DataFrame):
                concatenated = concatenated.squeeze()
            returns_dict[key] = concatenated.sort_index()
        else:
            returns_dict[key] = pd.Series(dtype=float)

    portfolios_df = pd.DataFrame(returns_dict)
    
    print(f"📈 Portfolio returns:")
    print(f"  Shape: {portfolios_df.shape}")
    print(f"  Date range: {portfolios_df.index.min()} to {portfolios_df.index.max()}")
    
    nan_counts = portfolios_df.isna().sum()
    if nan_counts.sum() > 0:
        print(f"  Missing values per portfolio:")
        for col in portfolios_df.columns:
            if nan_counts[col] > 0:
                print(f"    {col}: {nan_counts[col]} NaN")
    
    portfolios_df = portfolios_df.dropna(thresh=num_port//2)
    print(f"  After filtering: {portfolios_df.shape}\n")
    
    avg_turnover = {port: np.nanmean(values) if values else np.nan 
                    for port, values in turnover_values.items()}
    turnover_key = 'VW_turnover' if weighting else 'EW_turnover'
    
    turnover_df = pd.DataFrame({
        'portfolio': [int(p.replace('port','')) for p in avg_turnover.keys()],
        turnover_key: list(avg_turnover.values())
    })
    
    portfolios_stock_reallocation = {
        turnover_key: turnover_df,
        'reallocation': reallocation
    }

    return portfolios_df, portfolios_stock_reallocation


# ------------------------
# ESECUZIONE
# ------------------------
ELECTRE_portfolios, ELECTRE_portfolios_stock_reallocation = compute_electre_iii_rolling_strategy(
    df_norm, ELECTRE_configuration, weighting=False
) 

weighted_ELECTRE_portfolios, weighted_ELECTRE_portfolios_stock_reallocation = compute_electre_iii_rolling_strategy(
    df_norm, ELECTRE_configuration, weighting=True
)

portfolios['electre'] = ELECTRE_portfolios 
portfolios_stock_reallocation['electre'] = ELECTRE_portfolios_stock_reallocation 
weighted_portfolios['welectre'] = weighted_ELECTRE_portfolios 
portfolios_stock_reallocation['welectre'] = weighted_ELECTRE_portfolios_stock_reallocation


In [ ]:


# *************************************************************
# Questo per controllare che effettivamente electre funzioni **
# *************************************************************

## 1. Verifica del DataFrame dei Rendimenti (Risultato Principale)
print("--- 📊 Verifica Rendimenti (Prime 5 Righe) ---")
print(ELECTRE_portfolios.head())

print("\n--- 📈 Rendimenti Medi e Conteggio NaN per Portafoglio ---")
# Calcola la media e il numero di osservazioni non nulle
rendimenti_verificati = ELECTRE_portfolios.agg(['mean', 'count'])
print(rendimenti_verificati)

# ---
print("\n--- 📦 Verifica Composizione (Ribilanciamento) ---")
# Estrai il dizionario di ribilanciamento
reallocation_data = ELECTRE_portfolios_stock_reallocation['reallocation']

if not reallocation_data:
    print("ATTENZIONE: Il dizionario di ribilanciamento è vuoto. La computazione potrebbe non aver trovato abbastanza dati in nessuna finestra.")
else:
    # Prendi la prima data di ribilanciamento disponibile
    first_date = next(iter(reallocation_data['port1'].keys()))
    
    print(f"\nData di Ribilanciamento Controllata: {first_date}")
    
    # 2. Verifica del Portafoglio Migliore (port1)
    port1_holdings = reallocation_data['port1'][first_date]
    print(f"Portafoglio Long (Port1) - Totale Assets: {len(port1_holdings)}")
    print(f"Primi 5 Tickers: {port1_holdings[:5]}...")
    
    # 3. Verifica del Portafoglio Peggiore (portN)
    portN = f'port{ELECTRE_configuration["num_port"]}'
    portN_holdings = reallocation_data[portN][first_date]
    print(f"Portafoglio Short ({portN}) - Totale Assets: {len(portN_holdings)}")
    print(f"Primi 5 Tickers: {portN_holdings[:5]}...")

In [ ]:
# ========================== FACCIO I PORTAFOGLI ==========================

# PARTE 6 - COMPUTE EW-VW MCDM STRATEGY

# --- EW (Equal Weight) ---
from tqdm import tqdm

TOPSIS_portfolios, TOPSIS_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'topsis', MJ_configuration, weighting=False)
VIKOR_portfolios, VIKOR_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'vikor', MJ_configuration, weighting=False)
PROMETHEE_portfolios, PROMETHEE_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'promethee', MJ_configuration, weighting=False)
AHP_portfolios, AHP_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'ahp', MJ_configuration, weighting=False)

# inserisco EW mcdm in portfolios e reallocation
portfolios['topsis'] = TOPSIS_portfolios
portfolios['vikor'] = VIKOR_portfolios
portfolios['promethee'] = PROMETHEE_portfolios
portfolios['ahp'] = AHP_portfolios

portfolios_stock_reallocation['topsis'] = TOPSIS_portfolios_stock_reallocation
portfolios_stock_reallocation['vikor'] = VIKOR_portfolios_stock_reallocation
portfolios_stock_reallocation['promethee'] = PROMETHEE_portfolios_stock_reallocation
portfolios_stock_reallocation['ahp'] = AHP_portfolios_stock_reallocation

# --- VW (Value Weighted) ---
TOPSIS_weighted_portfolios, TOPSIS_weighted_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'topsis', MJ_configuration, weighting=True)
VIKOR_weighted_portfolios, VIKOR_weighted_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'vikor', MJ_configuration, weighting=True)
PROMETHEE_weighted_portfolios, PROMETHEE_weighted_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'promethee', MJ_configuration, weighting=True)
AHP_weighted_portfolios, AHP_weighted_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'ahp', MJ_configuration, weighting=True)

# inserisco VW mcdm in portfolios e reallocation
weighted_portfolios['wtopsis'] = TOPSIS_weighted_portfolios
weighted_portfolios['wvikor'] = VIKOR_weighted_portfolios
weighted_portfolios['wpromethee'] = PROMETHEE_weighted_portfolios
weighted_portfolios['wahp'] = AHP_weighted_portfolios

portfolios_stock_reallocation['wtopsis'] = TOPSIS_weighted_portfolios_stock_reallocation
portfolios_stock_reallocation['wvikor'] = VIKOR_weighted_portfolios_stock_reallocation
portfolios_stock_reallocation['wpromethee'] = PROMETHEE_weighted_portfolios_stock_reallocation
portfolios_stock_reallocation['wahp'] = AHP_weighted_portfolios_stock_reallocation


In [ ]:
# *************************************************************************************
# Questo per vedere quale portafoglio funzioni meglio solo con rendimento (P10 - EW) **
# *************************************************************************************

metodi = list(portfolios.keys())                                                    ## Qua modificare per vedere i vari cosi da mettere nella tabella

best_portfolios = {}                                                                                                              

print("Studio dei rendimenti dei portafogli *port10* (EW) per ogni metodo:")
for metodo in metodi:
    df_metodo = portfolios[metodo]

    if 'port10' not in df_metodo.columns:
        print(f"⚠️ Attenzione: {metodo} non ha port10.")
        continue

    rendimento_port10 = df_metodo['port10'].mean()

    best_portfolios[metodo] = {
        'portafoglio': 'port10',
        'rendimento_medio': rendimento_port10
    }

for metodo, info in best_portfolios.items():
    print(f"{metodo.upper():<10} ➜ rendimento medio mensile EW_P10 = {info['rendimento_medio']:.4f}")

# Trova il metodo il cui port10 ha avuto il rendimento medio più alto
migliore_assoluto = max(best_portfolios.items(), key=lambda x: x[1]['rendimento_medio'])

metodo_top = migliore_assoluto[0]
rendimento_top = migliore_assoluto[1]['rendimento_medio']

print("*" * 50)
print(f"WINNER: Il metodo con il miglior PORT10 è: {metodo_top.upper()} con rendimento medio mensile di {rendimento_top:.4f}")
print("*" * 50)

metodi_w = list(weighted_portfolios.keys())

# *************************************************************************************
# Questo per vedere quale portafoglio funzioni meglio solo con rendimento (P10 - VW) **
# *************************************************************************************

best_wportfolios = {}                                                                                                                  

print("Studio dei rendimenti dei portafogli *port10* (VW) per ogni metodo:")
for metodo in metodi_w:
    df_metodo = weighted_portfolios[metodo]

    if 'port10' not in df_metodo.columns:
        print(f"⚠️ Attenzione: {metodo} non ha port10.")
        continue

    rendimento_port10 = df_metodo['port10'].mean()

    best_wportfolios[metodo] = {
        'portafoglio': 'port10',
        'rendimento_medio': rendimento_port10
    }

for metodo, info in best_wportfolios.items():
    print(f"{metodo.upper():<10} ➜ rendimento medio mensile VW_P10 = {info['rendimento_medio']:.4f}")

# Trova il metodo il cui port10 ha avuto il rendimento medio più alto
migliore_assoluto = max(best_wportfolios.items(), key=lambda x: x[1]['rendimento_medio'])

metodo_top = migliore_assoluto[0]
rendimento_top = migliore_assoluto[1]['rendimento_medio']

print("*" * 50)
print(f"WINNER: Il metodo con il miglior PORT10 (VW) è: {metodo_top.upper()} con rendimento medio mensile di {rendimento_top:.4f}")
print("*" * 50)


In [ ]:
# ************************************************************************************
# Questo per vedere quale portafoglio funzioni meglio solo con rendimento (LS - EW) **
# ************************************************************************************

metodi = list(portfolios.keys())

best_portfolios_LS = {}                                                                                                                   #    <--------------------------------

print("Studio dei rendimenti dei portafogli *long_short* (EW) per ogni metodo:")
for metodo in metodi:
    df_metodo = portfolios[metodo]

    if 'long_short' not in df_metodo.columns:
        print(f"⚠️ Attenzione: {metodo} non ha long_short.")
        continue

    rendimento_LS = df_metodo['long_short'].mean()

    best_portfolios_LS[metodo] = {
        'portafoglio': 'long_short',
        'rendimento_medio': rendimento_LS
    }

for metodo, info in best_portfolios_LS.items():
    print(f"{metodo.upper():<10} ➜ rendimento medio mensile EW_LS = {info['rendimento_medio']:.4f}")

# Trova il metodo il cui LS ha avuto il rendimento medio più alto
migliore_assoluto_LS = max(best_portfolios_LS.items(), key=lambda x: x[1]['rendimento_medio'])

metodo_top_LS = migliore_assoluto_LS[0]
rendimento_top_LS = migliore_assoluto_LS[1]['rendimento_medio']

print("*" * 50)
print(f"WINNER: Il metodo con il miglior longshort è: {metodo_top_LS.upper()} con rendimento medio mensile di {rendimento_top_LS:.4f}")
print("*" * 50)

# UGUALE MA CON LONG SHORT
metodi_w = list(weighted_portfolios.keys())

# *************************************************************************************
# Questo per vedere quale portafoglio funzioni meglio solo con rendimento (LS - EW) **
# *************************************************************************************

best_wportfolios_WLS = {}                                                                                                                   #    <--------------------------------

print("Studio dei rendimenti dei portafogli *long_short* (VW) per ogni metodo:")
for metodo in metodi_w:
    df_metodo = weighted_portfolios[metodo]

    if 'long_short' not in df_metodo.columns:
        print(f"⚠️ Attenzione: {metodo} non ha long_short.")
        continue

    rendimento_WLS = df_metodo['long_short'].mean()

    best_wportfolios_WLS[metodo] = {
        'portafoglio': 'long_short',
        'rendimento_medio': rendimento_WLS
    }

for metodo, info in best_wportfolios_WLS.items():
    print(f"{metodo.upper():<10} ➜ rendimento medio mensile VW_LS = {info['rendimento_medio']:.4f}")

# Trova il metodo il cui LS ha avuto il rendimento medio più alto
migliore_assoluto_WLS = max(best_wportfolios_WLS.items(), key=lambda x: x[1]['rendimento_medio'])

metodo_top_WLS = migliore_assoluto_WLS[0]
rendimento_top_WLS = migliore_assoluto_WLS[1]['rendimento_medio']

print("*" * 50)
print(f"WINNER: Il metodo con il miglior longshort è: {metodo_top_WLS.upper()} con rendimento medio mensile di {rendimento_top_WLS:.4f}")
print("*" * 50)

In [ ]:
# ========================== STATISTICHE PORTAFOGLI (TUTTI) ==========================

def calculate_sharpe_ratio(returns_series, risk_free_rate=0.0):

    '''
    Calcola lo Sharpe Ratio annualizzato.
    Assume returns_series sono rendimenti mensili.
    '''

    mean_return = returns_series.mean() * 12 # Rendimento medio annualizzato
    std_dev = returns_series.std(ddof=1) * np.sqrt(12) # Deviazione standard campionaria per riproducibilita

    annualized_risk_free_rate = risk_free_rate * 12

    if std_dev == 0:
        return np.nan
    return (mean_return - annualized_risk_free_rate) / std_dev

def calculate_sortino_ratio(returns_series, mar_monthly: float = 0.0):
    """
    Sortino Ratio annualizzato.
    - returns_series: rendimenti MENSILI
    - mar_monthly: Minimum Acceptable Return MENSILE (default 0.0). 
      Se vuoi usarlo come risk-free mensile, passalo qui.
    """
    # Numeratore: extra-mean rispetto al MAR, annualizzato
    excess_mean_ann = (returns_series.mean() - mar_monthly) * 12

    # Downside deviation mensile: sqrt( mean( min(0, r - MAR)^2 ) )
    downside_diff = np.minimum(0.0, returns_series - mar_monthly)
    downside_dev_monthly = np.sqrt((downside_diff ** 2).mean())

    # Annualizzazione della downside deviation
    downside_dev_ann = downside_dev_monthly * np.sqrt(12)

    if downside_dev_ann == 0 or np.isnan(downside_dev_ann):
        return np.nan
    return excess_mean_ann / downside_dev_ann

def compute_portfolio_stats(best_portfolios, portfolios_dict, mar_monthly: float = 0.0):
    risultati = []

    for metodo, info in best_portfolios.items():
        portafoglio = info['portafoglio']
        serie = portfolios_dict[metodo][portafoglio]

        rendimento_medio = serie.mean()
        volatilita = serie.std()
        sharpe = calculate_sharpe_ratio(serie)  # usa risk_free mensile = 0.0 di default
        sortino = calculate_sortino_ratio(serie, mar_monthly=mar_monthly)

        # Calcolo drawdown
        cumulative = (1 + serie).cumprod()
        peak = cumulative.cummax()
        drawdown = (cumulative - peak) / peak
        max_drawdown = abs(drawdown.min())

        risultati.append({
            'Metodo': metodo,
            'Portafoglio': portafoglio,
            'Rendimento medio': rendimento_medio,
            'Volatilità': volatilita,
            'Sharpe Ratio': sharpe,
            'Sortino Ratio': sortino,
            'Max Drawdown': max_drawdown
        })

    return pd.DataFrame(risultati)

# **********************************************************************
# Tutte le caratteristiche di tutti i portafogli (qua P10 - EW e VW) **
# **********************************************************************

print("\n--- Statistiche dei portafogli basandosi su P10 ---")
stats_ew = compute_portfolio_stats(best_portfolios, portfolios)
stats_vw = compute_portfolio_stats(best_wportfolios, weighted_portfolios)

print("Statistiche portafogli Equal Weighted (EW):")
print(stats_ew.round(4))
print("\nStatistiche portafogli Value Weighted (VW):")
print(stats_vw.round(4))

# *********************************************************************
# Tutte le caratteristiche di tutti i portafogli (qua LS - EW e VW) **
# *********************************************************************

print("\n--- Statistiche dei portafogli basandosi su LS ---")
stats_ew_LS = compute_portfolio_stats(best_portfolios_LS, portfolios)
stats_vw_LS = compute_portfolio_stats(best_wportfolios_WLS, weighted_portfolios)

print("Statistiche portafogli Equal Weighted (EW):")
print(stats_ew_LS.round(4))
print("\nStatistiche portafogli Value Weighted (VW):")
print(stats_vw_LS.round(4))

In [ ]:
# ========================== TEST STATISTICI SU SHARPE - BOOTSTRAP A BLOCCHI ==========================

# **********************************************************************
# Cambiato da bootstrap a blocchi --> versione di Romano
# Bootstrap = singola stat, senza confronti multipli
# Romano = confronti multipli, controlla il data snooping (modificare i dati artificalmente per avere risultati statistici ok)
#           Robusto a non-normalita', standard nella letteratura
# **********************************************************************

def bootstrap_sharpe_comparison_block(returns1, returns2, num_bootstrap_samples=2000,

    block_size=6, risk_free_rate=0.0):
    """

    Confronta gli Sharpe Ratio di due serie di rendimenti usando il block bootstrap.

    Restituisce la proporzione di volte che Sharpe1 > Sharpe2 (p-value unilaterale).

    Args:

    returns1 (pd.Series): Serie di rendimenti del primo portafoglio.

    returns2 (pd.Series): Serie di rendimenti del secondo portafoglio.

    num_bootstrap_samples (int): Numero di simulazioni bootstrap da eseguire.

    block_size (int): La dimensione del blocco di rendimenti da campionare.

    risk_free_rate (float): Tasso di rendimento risk-free mensile.

    """

    sharpes_diff = []
    n = len(returns1)

    # Calcola il numero di blocchi disponibili
    num_blocks = n // block_size
    if num_blocks == 0:
        print("Errore: la dimensione del blocco è maggiore della lunghezza dei dati.")
        return np.nan


    for _ in range(num_bootstrap_samples):

    # *** MODIFICA: Campionamento con block bootstrap ***
    # 1. Campiona gli INDICI dei blocchi con reinserimento
        block_indices = np.random.choice(num_blocks, size=num_blocks, replace=True)

    # 2. Ricostruisci le serie di rendimenti concatenando i blocchi campionati

        sample1 = np.empty(0)
        sample2 = np.empty(0)
        for idx in block_indices:
            start_idx = idx * block_size
            end_idx = start_idx + block_size
            sample1 = np.concatenate([sample1, returns1.iloc[start_idx:end_idx]])
            sample2 = np.concatenate([sample2, returns2.iloc[start_idx:end_idx]])

        # Assicura che la lunghezza del campione sia esattamente la stessa dell'originale
        sample1 = sample1[:n]
        sample2 = sample2[:n]

        # *** FINE MODIFICA ***

        sharpe1 = calculate_sharpe_ratio(pd.Series(sample1), risk_free_rate)
        sharpe2 = calculate_sharpe_ratio(pd.Series(sample2), risk_free_rate)


        if not np.isnan(sharpe1) and not np.isnan(sharpe2):
            sharpes_diff.append(sharpe1 - sharpe2)


    if not sharpes_diff:
        return np.nan

    p_value_one_sided = np.sum(np.array(sharpes_diff) <= 0) / len(sharpes_diff)

    return p_value_one_sided

'''
Bootstrap di Romano (stazionario) 
'''
# Con questa parte genero una serie temporale bootstrap (blocchi lunghezza casuale, mantiene autocorrelazione (ok letteratura))
def stationary_bootstrap_series(series, p):
    """
    Stationary Bootstrap (Politis & Romano, 1994)
    - p = probability of starting a new block (equiv. 1/block_size)
    """
    n = len(series)
    indices = np.zeros(n, dtype=int)

    # 1. Scegli un punto di partenza a caso
    indices[0] = np.random.randint(0, n)

    # 2. Costruisci gli indici uno per uno
    for t in range(1, n):
        if np.random.rand() < p:
            # Inizio un nuovo blocco
            indices[t] = np.random.randint(0, n)
        else:
            # Continuo il blocco precedente (circolare)
            indices[t] = (indices[t-1] + 1) % n

    return series.iloc[indices].reset_index(drop=True)

#Genero due serie bootrsap indipendenti, calcolo sharpe e faccio differenza, calcolo p (prob che WMJ NON sia migliore) e siamo a posto
#                   ATTENZIONE QUA DENTROOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
def bootstrap_sharpe_comparison_stationary(
    returns1, returns2, num_bootstrap_samples=2000,
    block_size=6, risk_free_rate=0.0
):
    """
    Confronta gli Sharpe Ratio usando lo Stationary Bootstrap (Politis & Romano).
    """
    sharpes_diff = []
    n = len(returns1)

    # p = probabilità di aprire un nuovo blocco
    p = 1 / block_size   # CONSIGLIATO in letteratura

    for _ in range(num_bootstrap_samples):

        # --- Stationary bootstrap sample ---
        sample1 = stationary_bootstrap_series(returns1, p)          # Cosi serie indipendenti, portafogli costruiti su stesso perioo immagino siano dipendenti? 
        sample2 = stationary_bootstrap_series(returns2, p)
        # Calcolo Sharpe su serie bootstrappate
        sharpe1 = calculate_sharpe_ratio(sample1, risk_free_rate)
        sharpe2 = calculate_sharpe_ratio(sample2, risk_free_rate)

        if not np.isnan(sharpe1) and not np.isnan(sharpe2):
            sharpes_diff.append(sharpe1 - sharpe2)

    if not sharpes_diff:
        return np.nan

    # p-value unilaterale: P(Sharpe1 <= Sharpe2)
    sharpes_diff = np.array(sharpes_diff)
    p_value_one_sided = np.mean(sharpes_diff <= 0)

    return p_value_one_sided

'''
Fine Bootstrap di Romano (stazionario)
'''

# ******************************************
# Applicazione del bootstrap su P10 - VW **
# ******************************************

# --- Applicazione ---
print("\n--- Confronto Statistico degli Sharpe Ratio basandoci su P10 (WMJ vs Altri VW) ---")
# Tasso risk-free mensile (assumo 0 per semplicità ) --> giustificare durante tesi
RISK_FREE_MONTHLY = 0.0                                                                             # --> ATTENZIONE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

wmj_port_name = best_wportfolios['wmj']['portafoglio']
wmj_returns_series = weighted_portfolios['wmj'][wmj_port_name]

for metodo in [k for k in weighted_portfolios.keys() if k != 'wmj']:
    other_port_name = best_wportfolios[metodo]['portafoglio']
    other_returns_series = weighted_portfolios[metodo][other_port_name]

    min_len = min(len(wmj_returns_series), len(other_returns_series))
    wmj_sample_returns = wmj_returns_series.iloc[:min_len]
    other_sample_returns = other_returns_series.iloc[:min_len]

    # Chiamo la funzione (funzione vecchia)
    # p_value = bootstrap_sharpe_comparison_block(wmj_sample_returns, other_sample_returns, num_bootstrap_samples=2000, risk_free_rate=RISK_FREE_MONTHLY)

    #funzione di Romano
    p_value = bootstrap_sharpe_comparison_stationary(wmj_sample_returns, other_sample_returns, num_bootstrap_samples=2000, block_size=6, risk_free_rate=RISK_FREE_MONTHLY)

    # Interpretazione del p-value   
    alpha = 0.05 # Livello di significatività

    print(f"\nConfronto WMJ vs {metodo.upper()} - P10:")
    if p_value < alpha:
        print(f"La performance risk-adjusted di WMJ è STATISTICAMENTE SUPERIORE a quella di {metodo.upper()} (p-value = {p_value:.4f})")
    else:
        print(f"Non c'è una differenza statisticamente significativa nella performance risk-adjusted tra WMJ e {metodo.upper()} (p-value = {p_value:.4f})")


# ******************************************
# Applicazione del bootstrap su LS - VW **
# ******************************************

# --- Applicazione ---
print("\n--- Confronto Statistico degli Sharpe Ratio basandoci su LS (WMJ vs Altri VW) ---")
# Tasso risk-free mensile (assumo 0 per semplicità ) --> giustificare durante tesi
RISK_FREE_MONTHLY = 0.0

wmj_port_name = best_wportfolios_WLS['wmj']['portafoglio']
wmj_returns_series = weighted_portfolios['wmj'][wmj_port_name]

for metodo in [k for k in weighted_portfolios.keys() if k != 'wmj']:
    other_port_name = best_wportfolios_WLS[metodo]['portafoglio']
    other_returns_series = weighted_portfolios[metodo][other_port_name]

    min_len = min(len(wmj_returns_series), len(other_returns_series))
    wmj_sample_returns = wmj_returns_series.iloc[:min_len]
    other_sample_returns = other_returns_series.iloc[:min_len]

    # Chiamo la funzione (vecchia)
    # p_value = bootstrap_sharpe_comparison_block(wmj_sample_returns, other_sample_returns, num_bootstrap_samples=2000, risk_free_rate=RISK_FREE_MONTHLY)

    #funzione di Romano
    p_value = bootstrap_sharpe_comparison_stationary(wmj_sample_returns, other_sample_returns, num_bootstrap_samples=2000, block_size=6, risk_free_rate=RISK_FREE_MONTHLY)

    # Interpretazione del p-value   
    alpha = 0.05 # Livello di significatività

    print(f"\nConfronto WMJ vs {metodo.upper()} - LS:")
    if p_value < alpha:
        print(f"La performance risk-adjusted di WMJ è STATISTICAMENTE SUPERIORE a quella di {metodo.upper()} (p-value = {p_value:.4f})")
    else:
        print(f"Non c'è una differenza statisticamente significativa nella performance risk-adjusted tra WMJ e {metodo.upper()} (p-value = {p_value:.4f})")


# Dai risultati, wmj è tra i portafogli più efficienti (ottimo trade-off rendimento/rischio) 

In [ ]:
# ========================== TEST STATISTICO SU RENDIMENTI ==========================

from scipy.stats import ttest_rel, wilcoxon, shapiro
from statsmodels.stats.multitest import multipletests
import numpy as np
import pandas as pd

'''
t-test se dati normali
Wilcoxon se non normali
Cohens'd piccolo --> questo significa che, sebbene WMJ sia statisticamente superiore, la dimensione pratica della differenza non è enorme 9tipico dei dati finanziari che sono rumorosi)
Correzione Holm per test multipli (meno conservativa di Bonferroni) --> riduce falsi positivi mantenendo potenza

W --> wilcoxon statisticamente significativo
w --> wilconox non significativo
T --> t-test statisticamente significativo
t --> t-test non significativo
✅ --> test raccomandato risultato significativo

Differenze non significative in LS perché portafogli LS tendono a ridurre il rischio e livellare i rendimenti, perché il long e il short si compensano parzialmente.
    Questo spesso diminuisce le differenze medie tra WMJ e gli altri metodi, quindi il test statistico fatica a trovare significatività.
'''

def comprehensive_comparison_test(series_a, series_b, name_a="WMJ", name_b="OTHER", alpha=0.05):
    """
    Esegue test sia parametrici che non-parametrici per confrontare due serie.
    """
    # Allineamento
    idx = series_a.index.intersection(series_b.index)
    a = series_a.loc[idx].dropna()
    b = series_b.loc[idx].dropna()
    idx = a.index.intersection(b.index)
    a = a.loc[idx]
    b = b.loc[idx]
    
    if len(a) < 10:
        return {
            'n_observations': len(a),
            'warning': f"Troppo poche osservazioni ({len(a)})"
        }
    
    # Calcola differenze
    differences = a - b
    
    # Test di normalità
    _, normality_p = shapiro(differences) if len(differences) >= 3 else (None, None)
    is_normal = normality_p > 0.05 if normality_p is not None else False
    
    # T-test parametrico
    t_stat, t_p = ttest_rel(a, b)
    
    # Test non-parametrico di Wilcoxon (più robusto)
    try:
        w_stat, w_p = wilcoxon(differences, alternative='two-sided')
    except:
        w_stat, w_p = np.nan, np.nan
    
    # Statistiche descrittive
    mean_a = a.mean()
    mean_b = b.mean()
    mean_diff = differences.mean()
    median_diff = differences.median()
    std_diff = differences.std()
    
    # Effect size (Cohen's d) --> VECCHIO, UTILE PER CAMPIONI INDIPENDENTI (I NOSTRI LO SONO????)
    '''
    pooled_std = np.sqrt((a.var() + b.var()) / 2)
    cohens_d = mean_diff / pooled_std if pooled_std > 0 else np.nan
    '''

    # Nuovo Cohen che tiene conto di paired samples
    std_diff = differences.std(ddof=1)  # deviazione standard della differenza
    cohens_d = differences.mean() / std_diff

    
    return {
        'n_observations': len(a),
        'mean_a': mean_a,
        'mean_b': mean_b,
        'mean_difference': mean_diff,
        'median_difference': median_diff,
        'std_difference': std_diff,
        'cohens_d': cohens_d,
        'normality_p': normality_p,
        'is_normal': is_normal,
        't_statistic': t_stat,
        't_p_value': t_p,
        'wilcoxon_statistic': w_stat,
        'wilcoxon_p_value': w_p,
        't_significant': t_p < alpha,
        'wilcoxon_significant': w_p < alpha,
        'recommended_test': 'wilcoxon' if not is_normal else 't_test'
    }

def perform_multiple_comparisons_analysis(wmj_returns, other_portfolios, alpha=0.05, 
                                        correction_method='holm'):
    """
    Esegue test multipli con correzione per confronti multipli.
    """
    results = {}
    methods = [m for m in other_portfolios.keys() if m != 'wmj']
    
    # Raccogli tutti i p-value
    t_p_values = []
    w_p_values = []
    
    for method_name in methods:
        other_returns = other_portfolios[method_name]
        result = comprehensive_comparison_test(
            wmj_returns, other_returns, "WMJ", method_name.upper(), alpha
        )
        results[method_name] = result
        
        if 'warning' not in result:
            t_p_values.append(result['t_p_value'])
            w_p_values.append(result['wilcoxon_p_value'])
        else:
            t_p_values.append(1.0)  # Non significativo per metodi con warning
            w_p_values.append(1.0)
    
    # Correzione per test multipli
    t_corrected = multipletests(t_p_values, alpha=alpha, method=correction_method)
    w_corrected = multipletests(w_p_values, alpha=alpha, method=correction_method)
    
    # Aggiungi correzioni ai risultati
    for i, method_name in enumerate(methods):
        if 'warning' not in results[method_name]:
            results[method_name]['t_p_corrected'] = t_corrected[1][i]
            results[method_name]['wilcoxon_p_corrected'] = w_corrected[1][i]
            results[method_name]['t_significant_corrected'] = t_corrected[0][i]
            results[method_name]['wilcoxon_significant_corrected'] = w_corrected[0][i]
    
    return results, correction_method

def print_comprehensive_results(results, title, correction_method=None, alpha=0.05):
    """
    Stampa risultati comprensivi con raccomandazioni.
    """
    print(f"\n{'='*80}")
    print(f"{title}")
    print(f"{'='*80}")
    
    # Header
    header = f"{'Metodo':<12} {'n':<4} {'T-test':<8} {'Wilcoxon':<9} {'Raccomandato':<12} {'Effect Size':<11} {'Note'}"
    print(header)
    print("-" * len(header))
    
    significant_methods = []
    
    for method, result in results.items():
        if 'warning' in result:
            print(f"{method.upper():<12} {result['n_observations']:<4} {'N/A':<8} {'N/A':<9} {'N/A':<12} {'N/A':<11} {result['warning']}")
            continue
        
        method_label = method.upper()
        
        # Scegli il test raccomandato
        if result['recommended_test'] == 'wilcoxon':
            main_p = result['wilcoxon_p_corrected'] if correction_method else result['wilcoxon_p_value']
            is_significant = main_p < alpha
            test_symbol = "W" if is_significant else "w"
        else:
            main_p = result['t_p_corrected'] if correction_method else result['t_p_value']
            is_significant = main_p < alpha
            test_symbol = "T" if is_significant else "t"
        
        # Effect size interpretation
        d = abs(result['cohens_d'])
        if d >= 0.8:
            effect = "Grande"
        elif d >= 0.5:
            effect = "Medio"  
        elif d >= 0.2:
            effect = "Piccolo"
        else:
            effect = "Trascurabile"
        
        # Note
        note = ""
        if not result['is_normal']:
            note += "Non-norm"
        if result['mean_difference'] > 0:
            note += " WMJ>" + method_label[:3]
        else:
            note += " WMJ<" + method_label[:3]
        
        t_p_display = f"{result['t_p_value']:.4f}"
        w_p_display = f"{result['wilcoxon_p_value']:.4f}"
        sig_symbol = "✅" if is_significant else "➖"
        
        print(f"{method_label:<12} {result['n_observations']:<4} {t_p_display:<8} {w_p_display:<9} "
              f"{test_symbol+sig_symbol:<12} {effect:<11} {note}")
        
        if is_significant:
            significant_methods.append(method_label)
    
    print(f"\n🎯 WMJ significativamente superiore a: {significant_methods}")
    print(f"📊 Test raccomandato: Wilcoxon (dati non-normali)")
    if correction_method:
        print(f"🔧 Correzione applicata: {correction_method}")

# ==================== APPLICAZIONE ======================

alpha = 0.05

# P10 Analysis
print("ANALISI STATISTICA ROBUSTA: P10 VW")
wmj_ret_p10 = weighted_portfolios['wmj'][best_wportfolios['wmj']['portafoglio']]
other_portfolios_p10 = {
    method: weighted_portfolios[method][best_wportfolios[method]['portafoglio']]
    for method in weighted_portfolios.keys() if method != 'wmj'
}

results_p10, correction_method = perform_multiple_comparisons_analysis(
    wmj_ret_p10, other_portfolios_p10, alpha, 'holm'
)
print_comprehensive_results(results_p10, "P10 VW: WMJ vs Altri Metodi (Test Robusti)", correction_method, alpha)

# LS Analysis  
print("\n\nANALISI STATISTICA ROBUSTA: LS VW")
wmj_ret_ls = weighted_portfolios['wmj'][best_wportfolios_WLS['wmj']['portafoglio']]
other_portfolios_ls = {
    method: weighted_portfolios[method][best_wportfolios_WLS[method]['portafoglio']]
    for method in weighted_portfolios.keys() if method != 'wmj'
}

results_ls, correction_method = perform_multiple_comparisons_analysis(
    wmj_ret_ls, other_portfolios_ls, alpha, 'holm'
)
print_comprehensive_results(results_ls, "LS VW: WMJ vs Altri Metodi (Test Robusti)", correction_method, alpha)

# Summary insight
print(f"\n{'='*80}")
print("📈 RIEPILOGO INSIGHTS")
print(f"{'='*80}")
print("• I rendimenti finanziari raramente seguono distribuzione normale")
print("• Il test di Wilcoxon è più appropriato per questi dati") 
print("• La correzione di Holm controlla il tasso di errore di Tipo I nei test multipli")
print("• L'effect size (Cohen's d) quantifica la dimensione pratica delle differenze")
print("• WMJ sembra performare meglio nella strategia P10 che in Long-Short")

In [ ]:
# ========================== COSTI DI TRANSAZIONE ==========================

def comprehensive_transaction_cost_analysis_extended():
    """
    Analisi completa dei costi di transazione per P10 e Long-Short
    """
    
    # 1. Raccogli turnover e rendimenti per P10 VW
    turnover_data_p10 = {}
    gross_returns_p10 = {}
    
    for method in weighted_portfolios.keys():
        if method in portfolios_stock_reallocation:
            df_turn = portfolios_stock_reallocation[method]['VW_turnover']
            turnover_p10 = df_turn[df_turn['portfolio'] == 10.0]['VW_turnover'].values[0]
            turnover_data_p10[method] = turnover_p10
            gross_returns_p10[method] = weighted_portfolios[method]['port10'].mean()
    
    # 2. Raccogli turnover e rendimenti per Long-Short VW
    # Per LS assumiamo che il turnover sia la media tra P1 e P10 (o usa metrica specifica se disponibile)
    turnover_data_ls = {}
    gross_returns_ls = {}
    
    for method in weighted_portfolios.keys():
        if method in portfolios_stock_reallocation:
            df_turn = portfolios_stock_reallocation[method]['VW_turnover']
            # Turnover LS = media ponderata tra P1 e P10 (o puoi usare metrica diversa)
            turnover_p1 = df_turn[df_turn['portfolio'] == 1.0]['VW_turnover'].values[0]
            turnover_p10 = df_turn[df_turn['portfolio'] == 10.0]['VW_turnover'].values[0]
            turnover_data_ls[method] = (turnover_p1 + turnover_p10) / 2  # Media semplice
            gross_returns_ls[method] = weighted_portfolios[method]['long_short'].mean()
    
    # 3. Range di costi di transazione realistici
    transaction_costs = [0.001, 0.002, 0.005, 0.01, 0.015, 0.02]  # 0.1% - 2%
    
    # 4. Calcola rendimenti netti per P10
    results_p10 = []
    for cost in transaction_costs:
        for method in turnover_data_p10.keys():
            gross_ret = gross_returns_p10[method]
            turnover = turnover_data_p10[method]
            
            cost_impact = turnover * cost
            net_return = gross_ret - cost_impact
            
            gross_series = weighted_portfolios[method]['port10']
            net_series = gross_series - cost_impact
            net_sharpe = calculate_sharpe_ratio(net_series)
            
            results_p10.append({
                'Strategy': 'P10',
                'Method': method.upper(),
                'Transaction_Cost': cost,
                'Turnover': turnover,
                'Gross_Return': gross_ret,
                'Cost_Impact': cost_impact,
                'Net_Return': net_return,
                'Net_Sharpe': net_sharpe,
                'Return_Loss_pct': (cost_impact / gross_ret) * 100 if gross_ret != 0 else 0
            })
    
    # 5. Calcola rendimenti netti per Long-Short
    results_ls = []
    for cost in transaction_costs:
        for method in turnover_data_ls.keys():
            gross_ret = gross_returns_ls[method]
            turnover = turnover_data_ls[method]
            
            cost_impact = turnover * cost
            net_return = gross_ret - cost_impact
            
            gross_series = weighted_portfolios[method]['long_short']
            net_series = gross_series - cost_impact
            net_sharpe = calculate_sharpe_ratio(net_series)
            
            results_ls.append({
                'Strategy': 'Long-Short',
                'Method': method.upper(),
                'Transaction_Cost': cost,
                'Turnover': turnover,
                'Gross_Return': gross_ret,
                'Cost_Impact': cost_impact,
                'Net_Return': net_return,
                'Net_Sharpe': net_sharpe,
                'Return_Loss_pct': (cost_impact / gross_ret) * 100 if gross_ret != 0 else 0
            })
    
    # 6. Combina i risultati
    all_results = results_p10 + results_ls
    return pd.DataFrame(all_results)

# Esegui analisi estesa
cost_analysis_extended = comprehensive_transaction_cost_analysis_extended()

# 7. Ranking separato per P10 e Long-Short
def print_strategy_rankings(df, strategy_name, costs_to_show=[0.001, 0.005, 0.01, 0.02]):
    print(f"\n🏆 RANKING {strategy_name.upper()} - RENDIMENTO NETTO:")
    print("=" * 80)
    
    strategy_data = df[df['Strategy'] == strategy_name]
    
    for cost in costs_to_show:
        subset = strategy_data[strategy_data['Transaction_Cost'] == cost].copy()
        subset = subset.sort_values('Net_Return', ascending=False)
        
        print(f"\n📊 Costo transazione: {cost*100:.1f}%")
        print(f"{'Rank':<4} {'Method':<12} {'Net Return':<12} {'Turnover':<10} {'Loss %':<8}")
        print("-" * 50)
        
        for i, (_, row) in enumerate(subset.head(5).iterrows(), 1):
            print(f"{i:<4} {row['Method']:<12} {row['Net_Return']:.4f}      {row['Turnover']:.4f}    {row['Return_Loss_pct']:.2f}%")
        
        # Posizione di WMJ
        wmj_subset = subset[subset['Method'] == 'WMJ']
        if not wmj_subset.empty:
            wmj_pos = subset.reset_index().query("Method == 'WMJ'").index[0] + 1
            print(f"🎯 WMJ Position: #{wmj_pos}")

# Stampa ranking per entrambe le strategie
print_strategy_rankings(cost_analysis_extended, 'P10')
print_strategy_rankings(cost_analysis_extended, 'Long-Short')

# 8. Analisi break-even comparativa
def break_even_analysis(df, strategy_name):
    print(f"\n\n💰 BREAK-EVEN ANALYSIS - {strategy_name.upper()}:")
    print("=" * 50)
    
    strategy_data = df[df['Strategy'] == strategy_name]
    wmj_data = strategy_data[strategy_data['Method'] == 'WMJ']
    
    if wmj_data.empty:
        print(f"⚠️ WMJ non trovato per {strategy_name}")
        return
    
    wmj_gross = wmj_data['Gross_Return'].iloc[0]
    wmj_turnover = wmj_data['Turnover'].iloc[0]
    
    print(f"WMJ Turnover: {wmj_turnover:.4f}")
    print(f"WMJ Gross Return: {wmj_gross:.4f}")
    
    competitors = ['WTOPSIS', 'WVIKOR', 'WPROMETHEE', 'DOLVOL6', 'GP', 'ILL6']
    unique_methods = strategy_data['Method'].unique()
    
    for comp in competitors:
        if comp in unique_methods:
            comp_data = strategy_data[strategy_data['Method'] == comp]
            comp_gross = comp_data['Gross_Return'].iloc[0]
            comp_turnover = comp_data['Turnover'].iloc[0]
            
            if abs(comp_turnover - wmj_turnover) > 1e-6:
                breakeven_cost = (comp_gross - wmj_gross) / (comp_turnover - wmj_turnover)
                if breakeven_cost > 0:
                    print(f"vs {comp}: Break-even cost = {breakeven_cost*100:.2f}%")
                else:
                    print(f"vs {comp}: WMJ dominante (sempre superiore)")

# Esegui break-even per entrambe
break_even_analysis(cost_analysis_extended, 'P10')
break_even_analysis(cost_analysis_extended, 'Long-Short')

# 10. Tabelle finali comparative
def create_summary_table(df, strategy_name):
    strategy_data = df[df['Strategy'] == strategy_name]
    summary_costs = strategy_data[strategy_data['Transaction_Cost'].isin([0.001, 0.005, 0.01])]
    
    pivot_table = summary_costs.pivot(index='Method', columns='Transaction_Cost', values='Net_Return')
    pivot_table.columns = [f'Cost_{c*100:.1f}%' for c in pivot_table.columns]
    pivot_table = pivot_table.sort_values('Cost_0.5%', ascending=False)
    
    print(f"\n📋 SUMMARY TABLE - {strategy_name.upper()} Net Returns by Transaction Cost:")
    print(pivot_table.round(4))
    
    return pivot_table

# Crea tabelle per entrambe le strategie
summary_p10 = create_summary_table(cost_analysis_extended, 'P10')
summary_ls = create_summary_table(cost_analysis_extended, 'Long-Short')

# 11. Confronto diretto P10 vs Long-Short per WMJ
print("\n🔍 WMJ: P10 vs Long-Short Performance")
print("=" * 50)

wmj_comparison = cost_analysis_extended[cost_analysis_extended['Method'] == 'WMJ']
for cost in [0.001, 0.005, 0.01, 0.02]:
    cost_data = wmj_comparison[wmj_comparison['Transaction_Cost'] == cost]
    p10_row = cost_data[cost_data['Strategy'] == 'P10']
    ls_row = cost_data[cost_data['Strategy'] == 'Long-Short']
    
    if not p10_row.empty and not ls_row.empty:
        p10_net = p10_row['Net_Return'].iloc[0]
        ls_net = ls_row['Net_Return'].iloc[0]
        p10_turnover = p10_row['Turnover'].iloc[0]
        ls_turnover = ls_row['Turnover'].iloc[0]
        
        print(f"\nCosto {cost*100:.1f}%:")
        print(f"  P10:        Net Return = {p10_net:.4f}, Turnover = {p10_turnover:.4f}")
        print(f"  Long-Short: Net Return = {ls_net:.4f}, Turnover = {ls_turnover:.4f}")
        print(f"  Differenza: {ls_net - p10_net:.4f} {'(LS > P10)' if ls_net > p10_net else '(P10 > LS)'}")